# Final Project

_[Project prompt](https://docs.google.com/document/d/1uAUJGEUzfNj6OsWNAimnYCw7eKaHhMUfU1MTj9YwYw4/edit?usp=sharing), [grading rubric](https://docs.google.com/document/d/1UYkYtDtHJS2nzodDjJ3X3c2AdYNuj3ueki58p_hQGhA/edit)_


**Background:**

Imagine your apartment lease is ending at the close of the year, and the search for a new apartment is on. To narrow down potential neighborhoods, you've identified key criteria that matter to you.

**Goals:**

- quiet neighborhood (relating to dataset of `311 complaints`)
- a lot of greenery (relating to dataset of `2015 tree census`) 
- within budget (relating to dataset of `Zillow`)


## Project Setup（we can edit when going ahead）

In [1]:
# all import statements needed for the project
import pandas as pd
import geopandas as gpd
import requests
import re
import psycopg2
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String, Float, Index
import geoalchemy2
from datetime import datetime, timedelta

In [2]:
# any constants the project might need

# Application token for downloading NYC open data
app_token = 'NJflFVV2YiwlXmMlt4Y9jwTGO'

# API for the dataset
api_311 = 'https://data.cityofnewyork.us/resource/erm2-nwe9.json'
api_tree='https://data.cityofnewyork.us/resource/uvpi-gqnh.json'

# date range for the NYC_311
start_date_311 = datetime(2015, 1, 1)
end_date_311 = datetime(2023, 10, 31)

# file path for zipcode data and zillow data
Zipcode_DIR = "data"
Zipcode_SHAPEFILE = f"{Zipcode_DIR }/nyc_zipcodes.shp"
Zillow_DIR="data"
Zillow_file=f"{Zillow_DIR}/zillow_rent_data.csv"

crs="EPSG:4326"  # coordinate reference system

# (lat, lon)
NEW_YORK_BOX_COORDS = ((40.560445, -74.242330), (40.908524, -73.717047))
LGA_BOX_COORDS = ((40.763589, -73.891745), (40.778865, -73.854838))
JFK_BOX_COORDS = ((40.639263, -73.795642), (40.651376, -73.766264))
EWR_BOX_COORDS = ((40.686794, -74.194028), (40.699680, -74.165205))

## database information
username = 'postgres'
password = ''
host = 'localhost'
database = 'GroupProject'
DATABASE_URL=f'postgresql://{username}:{password}@{host}/{database}'
DATABASE_SCHEMA_FILE = "schema.sql"
QUERY_DIRECTORY = "queries"

In [3]:
# Make sure the QUERY_DIRECTORY exists
#try:
    #os.mkdir(QUERY_DIRECTORY)
#except Exception as e:
    #if e.errno == 17:
        # the directory already exists
        #pass
    #else:
        #raise

## Part 1: Data Preprocessing

### Download dataset of 311

In [4]:
## define a function which used to download online data
def download_data(query):
    headers = {'X-App-Token': app_token}
    response = requests.get(query, headers=headers)
    df = pd.read_json(response.text)
    
    return df

In [5]:
query_01 = f"{api_311}?$where=created_date between '2015-01-01T00:00:00' and '2023-10-31T00:00:00'&$limit=1000000"
query_02 = f"{api_311}?$where=created_date between '2015-01-01T00:00:00' and '2023-10-31T00:00:00'&$limit=1000000&$offset=1000000"
query_03 = f"{api_311}?$where=created_date between '2015-01-01T00:00:00' and '2023-10-31T00:00:00'&$limit=1000000&$offset=2000000"
query_04 = f"{api_311}?$where=created_date between '2015-01-01T00:00:00' and '2023-10-31T00:00:00'&$limit=1000000&$offset=3000000"
query_05 = f"{api_311}?$where=created_date between '2015-01-01T00:00:00' and '2023-10-31T00:00:00'&$limit=1000000&$offset=4000000"
query_06 = f"{api_311}?$where=created_date between '2015-01-01T00:00:00' and '2023-10-31T00:00:00'&$limit=1000000&$offset=5000000"
query_07 = f"{api_311}?$where=created_date between '2015-01-01T00:00:00' and '2023-10-31T00:00:00'&$limit=1000000&$offset=6000000"
query_08 = f"{api_311}?$where=created_date between '2015-01-01T00:00:00' and '2023-10-31T00:00:00'&$limit=1000000&$offset=7000000"
query_09 = f"{api_311}?$where=created_date between '2015-01-01T00:00:00' and '2023-10-31T00:00:00'&$limit=1000000&$offset=8000000"
query_10 = f"{api_311}?$where=created_date between '2015-01-01T00:00:00' and '2023-10-31T00:00:00'&$limit=1000000&$offset=9000000"
query_11 = f"{api_311}?$where=created_date between '2015-01-01T00:00:00' and '2023-10-31T00:00:00'&$limit=1000000&$offset=10000000"
query_12 = f"{api_311}?$where=created_date between '2015-01-01T00:00:00' and '2023-10-31T00:00:00'&$limit=1000000&$offset=11000000"
query_13 = f"{api_311}?$where=created_date between '2015-01-01T00:00:00' and '2023-10-31T00:00:00'&$limit=1000000&$offset=12000000"
query_14 = f"{api_311}?$where=created_date between '2015-01-01T00:00:00' and '2023-10-31T00:00:00'&$limit=1000000&$offset=13000000"
query_15 = f"{api_311}?$where=created_date between '2015-01-01T00:00:00' and '2023-10-31T00:00:00'&$limit=1000000&$offset=14000000"
query_16 = f"{api_311}?$where=created_date between '2015-01-01T00:00:00' and '2023-10-31T00:00:00'&$limit=1000000&$offset=15000000"
query_17 = f"{api_311}?$where=created_date between '2015-01-01T00:00:00' and '2023-10-31T00:00:00'&$limit=1000000&$offset=16000000"
query_18 = f"{api_311}?$where=created_date between '2015-01-01T00:00:00' and '2023-10-31T00:00:00'&$limit=1000000&$offset=17000000"
query_19 = f"{api_311}?$where=created_date between '2015-01-01T00:00:00' and '2023-10-31T00:00:00'&$limit=1000000&$offset=18000000"
query_20 = f"{api_311}?$where=created_date between '2015-01-01T00:00:00' and '2023-10-31T00:00:00'&$limit=1000000&$offset=19000000"
query_21 = f"{api_311}?$where=created_date between '2015-01-01T00:00:00' and '2023-10-31T00:00:00'&$limit=1000000&$offset=20000000"

In [7]:
query_10_data=download_data(query_10)
query_10_data.to_csv("query_10_data.csv",index=False)

### Download dataset of 2015 tree census

In [11]:
soql_query_tree_2015 = f"{api_tree}?$where=created_at between '2015-01-01T00:00:00' and '2015-12-31T23:59:59'&$limit=1000000000"
df_tree=download_data(soql_query_tree_2015 )
df_tree

,tree_id,block_id,created_at,tree_dbh,stump_diam,curb_loc,status,health,spc_latin,spc_common,...,boro_ct,state,latitude,longitude,x_sp,y_sp,council_district,census_tract,bin,bbl
0,180683,348711,2015-08-27,3,0,OnCurb,Alive,Fair,Acer rubrum,red maple,...,4073900,New York,40.723092,-73.844215,1.027431e+06,202756.7687,29.0,739.0,4052307.0,4.022210e+09
1,200540,315986,2015-09-03,21,0,OnCurb,Alive,Fair,Quercus palustris,pin oak,...,4097300,New York,40.794111,-73.818679,1.034456e+06,228644.8374,19.0,973.0,4101931.0,4.044750e+09
2,204026,218365,2015-09-05,3,0,OnCurb,Alive,Good,Gleditsia triacanthos var. inermis,honeylocust,...,3044900,New York,40.717581,-73.936608,1.001823e+06,200716.8913,34.0,449.0,3338310.0,3.028870e+09
3,204337,217969,2015-09-05,10,0,OnCurb,Alive,Good,Gleditsia triacanthos var. inermis,honeylocust,...,3044900,New York,40.713537,-73.934456,1.002420e+06,199244.2531,34.0,449.0,3338342.0,3.029250e+09
4,189565,223043,2015-08-30,21,0,OnCurb,Alive,Good,Tilia americana,American linden,...,3016500,New York,40.666778,-73.975979,9.909138e+05,182202.4260,39.0,165.0,3025654.0,3.010850e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
509520,155433,217978,2015-08-18,25,0,OnCurb,Alive,Good,Quercus palustris,pin oak,...,3051900,New York,40.713211,-73.954944,9.967407e+05,199121.6363,34.0,519.0,3062513.0,3.023690e+09
509521,183795,348185,2015-08-29,7,0,OnCurb,Alive,Good,Cladrastis kentukea,Kentucky yellowwood,...,4070700,New York,40.715194,-73.856650,1.023989e+06,199873.6475,29.0,707.0,4075448.0,4.031810e+09
509522,166161,401670,2015-08-22,12,0,OnCurb,Alive,Good,Acer rubrum,red maple,...,5020100,New York,40.620762,-74.136517,9.463514e+05,165466.0763,50.0,201.0,5011657.0,5.004080e+09
509523,184028,504204,2015-08-29,9,0,OnCurb,Alive,Good,Acer rubrum,red maple,...,2023502,New York,40.850828,-73.903115,1.011054e+06,249271.9507,15.0,23502.0,2007757.0,2.028120e+09


### Cleaning & filtering

#### 311 dataset

In [15]:
def get_311_data(dataset):
    ## create a list containing complaint type related to 'noise'
    distinct_complaint_types = list(dataset.complaint_type.unique()) ## find distict value of "complaint_type"
    
    noise_complaints = [complaint_type for complaint_type in distinct_complaint_types 
                    if "noise" in complaint_type.lower()] ## find distinct value realted to noise

    ## create a list containing all the columns we need
    selected_columns = ['unique_key', 'created_date',"complaint_type","incident_zip","latitude","longitude"]
    
    ## get dataframe only including "noise type" and columns we need
    noise_dataset=dataset[dataset.complaint_type.isin(noise_complaints)][selected_columns]
    
    ## convert data type of location to point
    noise_dataset['location'] = gpd.points_from_xy(noise_dataset.longitude, noise_dataset.latitude,crs="EPSG:4326")
    
    ## Drop latitude and longitude columns
    noise_dataset = noise_dataset.drop(['latitude', 'longitude'], axis=1)  
    
    ## rename some column
    noise_dataset=noise_dataset.rename(columns={"incident_zip":"zipcode"})
    
    return gpd.GeoDataFrame(noise_dataset, geometry='location')

In [ ]:
data_311=get_311_data(data_311_5)

In [87]:
data_311

,unique_key,created_date,complaint_type,zipcode,latitude,longitude,location
3650,29825424,2015-01-30T23:59:58.000,Noise - Residential,10467,40.879507,-73.866197,POINT (-73.86620 40.87951)
3652,29824625,2015-01-30T23:58:32.000,Noise - Residential,10009,40.728603,-73.978651,POINT (-73.97865 40.72860)
3654,29824492,2015-01-30T23:57:27.000,Noise - Commercial,10022,40.762856,-73.971933,POINT (-73.97193 40.76286)
3656,29827218,2015-01-30T23:57:07.000,Noise - Residential,10453,40.853689,-73.903955,POINT (-73.90396 40.85369)
3657,29825542,2015-01-30T23:57:04.000,Noise - Street/Sidewalk,10032,40.839404,-73.937622,POINT (-73.93762 40.83940)
...,...,...,...,...,...,...,...
385290,29833096,2015-02-01T00:01:52.000,Noise - Residential,11221,40.691681,-73.935593,POINT (-73.93559 40.69168)
385291,29835915,2015-02-01T00:01:07.000,Noise - Residential,10029,40.790059,-73.947885,POINT (-73.94788 40.79006)
385293,29835796,2015-02-01T00:00:35.000,Noise - Residential,10463,40.884083,-73.909506,POINT (-73.90951 40.88408)
385294,29831096,2015-02-01T00:00:19.000,Noise - Commercial,11423,40.708729,-73.778989,POINT (-73.77899 40.70873)


#### 2015 tree census dataset

In [88]:
def get_tree_data(dataset):
    selected_columns = ['tree_id', 'status','health','spc_common','zipcode','latitude', 'longitude']
    tree_data=dataset[selected_columns]
    
    tree_data['location'] = gpd.points_from_xy(tree_data.longitude, tree_data.latitude,crs="EPSG:4326")
    
    ## Drop latitude and longitude columns
    tree_data = tree_data.drop(['latitude', 'longitude'], axis=1)  
    
    tree_data=tree_data.rename(columns={"spc_common":"species"})
    
    return gpd.GeoDataFrame(tree_data, geometry='location')

data_tree=get_tree_data(df_tree)
data_tree

/var/folders/6k/mm4rw52d5r34tr4vw4278rk00000gn/T/ipykernel_98555/4278032873.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tree_data['location'] = gpd.points_from_xy(tree_data.longitude, tree_data.latitude,crs="EPSG:4326")


,tree_id,status,health,species,zipcode,location
0,180683,Alive,Fair,red maple,11375,POINT (-73.84422 40.72309)
1,200540,Alive,Fair,pin oak,11357,POINT (-73.81868 40.79411)
2,204026,Alive,Good,honeylocust,11211,POINT (-73.93661 40.71758)
3,204337,Alive,Good,honeylocust,11211,POINT (-73.93446 40.71354)
4,189565,Alive,Good,American linden,11215,POINT (-73.97598 40.66678)
...,...,...,...,...,...,...
509520,155433,Alive,Good,pin oak,11211,POINT (-73.95494 40.71321)
509521,183795,Alive,Good,Kentucky yellowwood,11375,POINT (-73.85665 40.71519)
509522,166161,Alive,Good,red maple,10314,POINT (-74.13652 40.62076)
509523,184028,Alive,Good,red maple,10457,POINT (-73.90311 40.85083)


#### zipcode dataset

In [17]:
def get_geo_data(filename):
    try:
        data = gpd.read_file(filename)
        
        #choose the columns we need
        selected_columns = ['ZIPCODE', 'COUNTY','geometry']
        data_nyc=data[selected_columns]
        
        ## rename some column
        zipcode_nyc=data_nyc.rename(columns={"ZIPCODE":"zipcode","COUNTY":"county","geometry":"location"})  
        zipcode_nyc=gpd.GeoDataFrame(zipcode_nyc, geometry='location') 
        zipcode_nyc = zipcode_nyc.to_crs(epsg=4326)
        
        return zipcode_nyc
    except:
        print(f"Could not read file: {filename}")

In [79]:
# get dataset we need
data_zipcode=get_geo_data(Zipcode_SHAPEFILE)
data_zipcode

,zipcode,county,location
0,11436,Queens,"POLYGON ((-73.80585 40.68291, -73.80569 40.682..."
1,11213,Kings,"POLYGON ((-73.93740 40.67973, -73.93487 40.679..."
2,11212,Kings,"POLYGON ((-73.90294 40.67084, -73.90223 40.668..."
3,11225,Kings,"POLYGON ((-73.95797 40.67066, -73.95576 40.670..."
4,11218,Kings,"POLYGON ((-73.97208 40.65060, -73.97192 40.650..."
...,...,...,...
258,10310,Richmond,"POLYGON ((-74.12065 40.64104, -74.12057 40.641..."
259,11693,Kings,"POLYGON ((-73.84076 40.62536, -73.84306 40.627..."
260,11249,Kings,"POLYGON ((-73.95805 40.72442, -73.95772 40.724..."
261,10162,New York,"POLYGON ((-73.95133 40.76931, -73.95165 40.769..."


#### zillow dataset

In [21]:
def get_zillow_data(filename):
    data=pd.read_csv(filename)
    
    ## only get the data of "NY" state
    zillow_data_NY=data[data.City=="New York"]
    
    ## create a column, which is the average price for each zipcode
    all_columns=zillow_data_NY.columns.tolist() ## get all column name
    pattern = re.compile(r'\d') ## to get date columns
    date_list=[i for i in all_columns if pattern.search(i)] # get date list
    avg_price=zillow_data_NY[date_list].mean(axis=1) #calculate avg price for each zipcode
    zillow_data_NY_with_avg = pd.concat([zillow_data_NY, avg_price.rename('avg_price')], axis=1)
    
    ## get dateframe contianing columns we need
    selected_columns=["RegionName","City","CountyName","avg_price"]
    zillow_NY_dataset=zillow_data_NY_with_avg[selected_columns]
    
    ## rename some column
    zillow_NY_dataset = zillow_NY_dataset.rename(columns={"RegionName": "zipcode","City":"city","CountyName":"countyname"})
    
    return zillow_NY_dataset

In [81]:
data_zillow=get_zillow_data(Zillow_file)
data_zillow

,zipcode,city,countyname,avg_price
4,11385,New York,Queens County,2476.923402
6,11208,New York,Kings County,2560.469803
12,11236,New York,Kings County,2323.980013
13,10467,New York,Bronx County,1861.072982
14,11373,New York,Queens County,2084.549636
...,...,...,...,...
6550,10282,New York,New York County,7029.339124
6561,11109,New York,Queens County,3518.929190
6644,10006,New York,New York County,3489.762997
6695,10162,New York,New York County,4968.739544


## Part 2: Storing Data

- main goal: taking the data from part 1 and populating a PostgreSQL database with tables generated from the datasets

### Create Database and Table

In [17]:
## first step: create database 'GroupProject'in PostgreSQL
!createdb GroupProject

createdb: error: database creation failed: ERROR:  database "GroupProject" already exists


In [18]:
!psql --dbname GroupProject -c 'CREATE EXTENSION postgis;'

ERROR:  extension "postgis" already exists


In [23]:
def create_tables(engine):
    metadata = MetaData()

    nyc_311 = Table(
        'NYC_311', metadata,
        Column('unique_key', Integer, primary_key=True),
        Column('created_date', String),
        Column('complaint_type', String),
        Column('zipcode', Integer),
        Column('location', Geometry('POINT', srid=4326))
    )

    nyc_tree = Table(
        'NYC_tree', metadata,
        Column('tree_id', Integer, primary_key=True),
        Column('status', String),
        Column('health', String),
        Column('species', String),
        Column('zipcode', Integer),
        Column('location', Geometry('POINT', srid=4326))
    )

    nyc_zipcode = Table(
        'NYC_zipcode', metadata,
        Column('zipcode', Integer, primary_key=True),
        Column('county', String),
        Column('location', Geometry('POLYGON', srid=4326))
    )

    nyc_zillow = Table(
        'NYC_zillow', metadata,
        Column('zipcode', Integer, primary_key=True),
        Column('city', String),
        Column('countyname', String),
        Column('avg_price', Float)
    )

    metadata.create_all(engine)

def create_indexes(connection):
    sql_create_indexes = """
    CREATE INDEX IF NOT EXISTS idx_nyc_311_location ON "NYC_311" USING gist (location);
    CREATE INDEX IF NOT EXISTS idx_nyc_tree_location ON "NYC_tree" USING gist (location);
    CREATE INDEX IF NOT EXISTS idx_nyc_zipcode_location ON "NYC_zipcode" USING gist (location);
    """

    with connection.cursor() as cursor:
        cursor.execute(sql_create_indexes)

In [24]:
engine = create_engine(DATABASE_URL)
create_tables(engine)

with psycopg2.connect(DATABASE_URL) as connection:
    create_indexes(connection)

NameError: name 'Geometry' is not defined

### Write Dataframe to Table

In [ ]:
def write_dataframes_to_table(df_name, table_name):
    engine = create_engine(DATABASE_URL)
    
    # if it cantains geometry data, use "gpd.to_postgis", else use "pd.to_sql"
    try:
        df_name.to_postgis(table_name, engine, if_exists='replace', index=False)
    except:
        df_name.to_sql(table_name, engine, if_exists='replace', index=False)

In [ ]:
write_dataframes_to_table(data_311, 'nyc_311')

In [ ]:
write_dataframes_to_table(data_tree, 'nyc_tree')

In [ ]:
write_dataframes_to_table(data_zipcode,"nyc_zipcode")

In [ ]:
write_dataframes_to_table(data_zillow,"nyc_zillow")

## Part 3: Understanding Data

### Query 1: Which area might be more calm to live in?

### Query 2: Where has the most greenery?

### Query 3: Can I afford a place in the areas with the most trees?

### Query 4: Could there be a correlation between an area’s rent, the number of its trees, and the number of 311 complaints?

### Query 5: Where has the most greenery (take 2)?

### Query 6: What is the immediate area like?

## Part 3: Understanding the Data

In [ ]:
# Helper function to write the queries to file
def write_query_to_file(query, outfile):
    raise NotImplementedError()

### Query 1

In [ ]:
QUERY_1_FILENAME = ""

QUERY_1 = """
TODO
"""

In [ ]:
engine.execute(QUERY_1).fetchall()

In [ ]:
write_query_to_file(QUERY_1, QUERY_1_FILENAME)

## Part 4: Visualizing the Data

### Visualization 1

In [ ]:
# use a more descriptive name for your function
def plot_visual_1(dataframe):
    figure, axes = plt.subplots(figsize=(20, 10))
    
    values = "..."  # use the dataframe to pull out values needed to plot
    
    # you may want to use matplotlib to plot your visualizations;
    # there are also many other plot types (other 
    # than axes.plot) you can use
    axes.plot(values, "...")
    # there are other methods to use to label your axes, to style 
    # and set up axes labels, etc
    axes.set_title("Some Descriptive Title")
    
    plt.show()

In [ ]:
def get_data_for_visual_1():
    # Query SQL database for the data needed.
    # You can put the data queried into a pandas dataframe, if you wish
    raise NotImplementedError()

In [ ]:
some_dataframe = get_data_for_visual_1()
plot_visual_1(some_dataframe)